In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.head(1)

,id,species,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,1,Acer_Opalus,0.007812,0.023438,0.023438,0.003906,0.011719,0.009766,0.027344,0.0,...,0.007812,0.0,0.00293,0.00293,0.035156,0.0,0.0,0.004883,0.0,0.025391


In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import StratifiedShuffleSplit

/Library/Python/2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
le = LabelEncoder().fit(train.species)
labels = le.transform(train.species)
classes = le.classes_
test_ids = test.id
train = train.drop(['species', 'id'], axis = 1)
test = test.drop(['id'], axis=1)


In [7]:
sss = StratifiedShuffleSplit(labels, 10, test_size=0.1, random_state=23)

for train_index, test_index in sss:
    X_train, X_test = train.values[train_index], train.values[test_index]
    Y_train, Y_test = labels[train_index], labels[test_index]


In [8]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [7]:
clf = LinearDiscriminantAnalysis()
clf.fit(X_train, Y_train)
predictions = clf.predict(X_test)

/Library/Python/2.7/site-packages/sklearn/discriminant_analysis.py:455: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/Library/Python/2.7/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [12]:
from sklearn.metrics import accuracy_score, log_loss

In [9]:
acc = accuracy_score(Y_test, predictions)
print("Accuracy: {:.4%}".format(acc))

Accuracy: 97.9798%


In [10]:
from sklearn.neural_network import MLPClassifier

In [11]:
clfNN = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(192, 192), random_state=1)
clfNN.fit(X_train, Y_train)
predictionsNN = clfNN.predict(X_test)

/Library/Python/2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


In [12]:
accNN = accuracy_score(Y_test, predictionsNN)
print("Accuracy: {:.4%}".format(accNN))


Accuracy: 92.4242%


In [9]:
#starting with keras NN
#One hot encoding for labels
from sklearn.preprocessing import StandardScaler
from keras.utils.np_utils import to_categorical


labelOneHot = to_categorical(Y_train)
labelOneHotTest = to_categorical(Y_test)
# Standardising the data to give zero mean
X_NN = StandardScaler().fit(X_train).transform(X_train)
X_testNN = StandardScaler().fit(X_test).transform(X_test)
print(X_NN.shape)
print labelOneHot.shape
print X_train.shape

Using TensorFlow backend.


(891, 192)
(891, 99)
(891, 192)


In [32]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation


In [86]:
#import keras.layers.convolutional as conv
model = Sequential()
#model.add(conv.Convolution1D(64,3,border_mode='same', input_shape=(192, 1024)))
model.add(Dense(1500,input_dim=192,  init='glorot_uniform', activation='relu'))
#model.add(Dense(1024, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(700, activation='sigmoid'))
#model.add(Dropout(0.5))
model.add(Dense(99, activation='softmax'))

In [90]:
#using all the training data for NN
model.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics = ["accuracy"])
filepath="weights.best.hdf5"

from keras.callbacks import ModelCheckpoint
#using keras callback function to save best model during epochs
#checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
#callbacks_list = [checkpoint]
#history = model.fit(X_NN,labelOneHot,batch_size=192,nb_epoch=120,callbacks=callbacks_list, verbose=0, validation_split=0.1)
history = model.fit(X_NN,labelOneHot,batch_size=80,nb_epoch=150, verbose=0, validation_split=0.1)


In [91]:
print('val_acc: ',min(history.history['val_acc']))
print('val_loss: ',min(history.history['val_loss']))
print('acc: ',max(history.history['acc']))
print('loss: ',min(history.history['loss']))
scores = model.evaluate(X_testNN, labelOneHotTest, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

('val_acc: ', 1.0)
('val_loss: ', 0.0059320637956261635)
('acc: ', 1.0)
('loss: ', 1.1920928955078125e-07)
acc: 99.49%


In [55]:
#loading our best saved model
modelLoad = Sequential()
modelLoad.add(Dense(1024,input_dim=192,  init='uniform', activation='relu'))
modelLoad.add(Dropout(0.3))
modelLoad.add(Dense(512, activation='sigmoid'))
modelLoad.add(Dropout(0.3))
modelLoad.add(Dense(99, activation='softmax'))
modelLoad.load_weights("weights.best.hdf5")
modelLoad.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics = ["accuracy"])
modelLoad = modelLoad.evaluate(X_testNN, labelOneHotTest, verbose=0)
print(" %.2f%%" % ( modelLoad[1]*100))
#this did not increase accuracy,so not using it

 99.49%


In [10]:
testNN = StandardScaler().fit(test).transform(test)
predictionsNNprob = model.predict_proba(testNN)

submission = pd.DataFrame(predictionsNNprob, columns=classes)
submission.insert(0, 'id', test_ids)
submission.reset_index()
# Export Submission
submission.to_csv('submission.csv', index = False)
submission.tail()

NameError: name 'model' is not defined

In [47]:
submission.shape

(594, 100)

In [74]:
#testing adaboost
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
clf2 =  AdaBoostClassifier()
clf.fit(X_train,Y_train)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

In [75]:
predictions = clf.predict(X_test)
acc = accuracy_score(Y_test, predictions)
print("Accuracy: {:.4%}".format(acc))

Accuracy: 97.9798%


In [95]:
#using regression
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver='lbfgs',multi_class='multinomial', C=1000, tol=0.0008)
clf.fit(X_NN, Y_train)
predictions = clf.predict(X_testNN)
acc = accuracy_score(Y_test, predictions)
print("Accuracy: {:.4%}".format(acc))
y_test = clf.predict_proba(testNN)

submission = pd.DataFrame(y_test, index=test_ids, columns=le.classes_)
#submission.to_csv('submission_log_reg.csv')

Accuracy: 99.4949%


In [99]:
from sklearn import linear_model
clfL = linear_model.SGDClassifier()
clfL.fit(X_NN, Y_train)
predictions = clfL.predict(X_testNN)
acc = accuracy_score(Y_test, predictions)
print("Accuracy: {:.4%}".format(acc))

Accuracy: 97.4747%


In [11]:
from xgboost import XGBClassifier
modelX = XGBClassifier()
modelX.fit(X_NN, Y_train)
predictions = modelX.predict(X_testNN)
acc = accuracy_score(Y_test, predictions)
print("Accuracy: {:.4%}".format(acc))

NameError: name 'accuracy_score' is not defined

In [13]:
acc = accuracy_score(Y_test, predictions)
print("Accuracy: {:.4%}".format(acc))

Accuracy: 86.8687%
